In [ ]:
from inference_sdk import InferenceHTTPClient

import cv2
import matplotlib.pyplot as plt

def draw_predictions_on_image(result, image_path):
    """
    將 Roboflow 偵測結果畫在圖片上並顯示出來。

    參數:
        result: dict，Roboflow API 回傳的偵測結果
        image_path: str，圖片的檔案路徑
    """
    # 載入並轉換圖片
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 畫出所有預測框
    for pred in result.get('predictions', []):
        x_center = pred['x']
        y_center = pred['y']
        width = pred['width']
        height = pred['height']
        conf = pred['confidence']
        label = pred['class']

        # 計算框的左上和右下座標
        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        # 畫框與標籤
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f"{label[:25]} {conf:.2f}", (x1, y1 - 8),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)

    # 顯示圖片
    plt.figure(figsize=(10, 8))
    plt.imshow(image)
    plt.axis('off')
    plt.title("Detection Result")
    plt.show()


CLIENT = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key="VXuipeVYwSTXxm4cRsrm"
)

result = CLIENT.infer('/content/彩色中央區域.png', model_id="baseball2-0d2jq/2")
draw_predictions_on_image(result, '/content/彩色中央區域.png')
# 現在這個程式的功能是 輸入圖片 他能回傳偵測並標出框框的圖片
# 幫我改成輸入影片回傳偵測並標出框框的影片

In [ ]:
from inference_sdk import InferenceHTTPClient
import cv2
import os
from google.colab.patches import cv2_imshow

# 初始化 Roboflow Client
CLIENT = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key="VXuipeVYwSTXxm4cRsrm"
)

def draw_predictions_on_frame(frame, predictions, offset_x=0, offset_y=0):
    """
    在 frame 上畫出 Roboflow 偵測框（含偏移補正）。
    """
    for pred in predictions:
        x_center = pred['x']
        y_center = pred['y']
        width = pred['width']
        height = pred['height']
        conf = pred['confidence']
        label = pred['class']

        x1 = int(x_center - width / 2 + offset_x)
        y1 = int(y_center - height / 2 + offset_y)
        x2 = int(x_center + width / 2 + offset_x)
        y2 = int(y_center + height / 2 + offset_y)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{label[:25]} {conf:.2f}", (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
    return frame

def detect_and_show_video(input_path, crop_ratio=1.0):
    """
    即時顯示 Roboflow 偵測結果（可指定中央區域比例）。
    """
    cap = cv2.VideoCapture(input_path)
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        h, w = frame.shape[:2]
        crop_w = int(w * crop_ratio)
        crop_h = int(h * crop_ratio)
        x1 = (w - crop_w) // 2
        y1 = (h - crop_h) // 2
        x2 = x1 + crop_w
        y2 = y1 + crop_h
        cropped = frame[y1:y2, x1:x2]
        cv2_imshow(cropped)

        # 儲存暫存圖片
        temp_path = "temp_frame.jpg"
        cv2.imwrite(temp_path, cropped)

        # Roboflow 偵測
        result = CLIENT.infer(temp_path, model_id="baseball2-0d2jq/2")

        # 在原圖上畫框（補上偏移）
        frame = draw_predictions_on_frame(frame, result.get('predictions', []), offset_x=x1, offset_y=y1)

        # 顯示畫面
        cv2_imshow(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1
        print(f"Processed frame: {frame_count}")

    cap.release()
    cv2.destroyAllWindows()
    os.remove(temp_path)
    print("Done!")

# 使用範例
# crop_ratio=1.0 偵測整張圖，crop_ratio=0.5 偵測中央一半大小的區域
detect_and_show_video('/content/pitch_0001.mp4', crop_ratio=0.4)